# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f07fcb97850>
├── 'a' --> tensor([[-0.9243, -0.2126, -1.3020],
│                   [ 1.5325,  0.0052, -1.1748]])
└── 'x' --> <FastTreeValue 0x7f07fcb978b0>
    └── 'c' --> tensor([[-0.7799,  0.6385, -0.5867,  1.0679],
                        [-0.9532,  0.0329, -0.7299,  1.0554],
                        [-0.6603, -0.0979,  0.8918,  1.2448]])

In [4]:
t.a

tensor([[-0.9243, -0.2126, -1.3020],
        [ 1.5325,  0.0052, -1.1748]])

In [5]:
%timeit t.a

62.9 ns ± 0.112 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f07fcb97850>
├── 'a' --> tensor([[-0.5844,  2.2411,  0.2170],
│                   [ 0.2634,  0.6950, -0.5526]])
└── 'x' --> <FastTreeValue 0x7f07fcb978b0>
    └── 'c' --> tensor([[-0.7799,  0.6385, -0.5867,  1.0679],
                        [-0.9532,  0.0329, -0.7299,  1.0554],
                        [-0.6603, -0.0979,  0.8918,  1.2448]])

In [7]:
%timeit t.a = new_value

67.5 ns ± 0.107 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9243, -0.2126, -1.3020],
               [ 1.5325,  0.0052, -1.1748]]),
    x: Batch(
           c: tensor([[-0.7799,  0.6385, -0.5867,  1.0679],
                      [-0.9532,  0.0329, -0.7299,  1.0554],
                      [-0.6603, -0.0979,  0.8918,  1.2448]]),
       ),
)

In [10]:
b.a

tensor([[-0.9243, -0.2126, -1.3020],
        [ 1.5325,  0.0052, -1.1748]])

In [11]:
%timeit b.a

57.7 ns ± 0.168 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3898, -0.4757,  1.0591],
               [ 0.4283, -0.1006,  0.7333]]),
    x: Batch(
           c: tensor([[-0.7799,  0.6385, -0.5867,  1.0679],
                      [-0.9532,  0.0329, -0.7299,  1.0554],
                      [-0.6603, -0.0979,  0.8918,  1.2448]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 4.16 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

807 ns ± 0.282 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 527 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 527 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0747e4fdf0>
├── 'a' --> tensor([[[-0.9243, -0.2126, -1.3020],
│                    [ 1.5325,  0.0052, -1.1748]],
│           
│                   [[-0.9243, -0.2126, -1.3020],
│                    [ 1.5325,  0.0052, -1.1748]],
│           
│                   [[-0.9243, -0.2126, -1.3020],
│                    [ 1.5325,  0.0052, -1.1748]],
│           
│                   [[-0.9243, -0.2126, -1.3020],
│                    [ 1.5325,  0.0052, -1.1748]],
│           
│                   [[-0.9243, -0.2126, -1.3020],
│                    [ 1.5325,  0.0052, -1.1748]],
│           
│                   [[-0.9243, -0.2126, -1.3020],
│                    [ 1.5325,  0.0052, -1.1748]],
│           
│                   [[-0.9243, -0.2126, -1.3020],
│                    [ 1.5325,  0.0052, -1.1748]],
│           
│                   [[-0.9243, -0.2126, -1.3020],
│                    [ 1.5325,  0.0052, -1.1748]]])
└── 'x' --> <FastTreeValue 0x7f0747e4fa00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 70.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f07fcb92190>
├── 'a' --> tensor([[-0.9243, -0.2126, -1.3020],
│                   [ 1.5325,  0.0052, -1.1748],
│                   [-0.9243, -0.2126, -1.3020],
│                   [ 1.5325,  0.0052, -1.1748],
│                   [-0.9243, -0.2126, -1.3020],
│                   [ 1.5325,  0.0052, -1.1748],
│                   [-0.9243, -0.2126, -1.3020],
│                   [ 1.5325,  0.0052, -1.1748],
│                   [-0.9243, -0.2126, -1.3020],
│                   [ 1.5325,  0.0052, -1.1748],
│                   [-0.9243, -0.2126, -1.3020],
│                   [ 1.5325,  0.0052, -1.1748],
│                   [-0.9243, -0.2126, -1.3020],
│                   [ 1.5325,  0.0052, -1.1748],
│                   [-0.9243, -0.2126, -1.3020],
│                   [ 1.5325,  0.0052, -1.1748]])
└── 'x' --> <FastTreeValue 0x7f07fcbf7fd0>
    └── 'c' --> tensor([[-0.7799,  0.6385, -0.5867,  1.0679],
                        [-0.9532,  0.0329, -0.7299,  1.0554],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 56.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.6 µs ± 96.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9243, -0.2126, -1.3020],
                [ 1.5325,  0.0052, -1.1748]],
       
               [[-0.9243, -0.2126, -1.3020],
                [ 1.5325,  0.0052, -1.1748]],
       
               [[-0.9243, -0.2126, -1.3020],
                [ 1.5325,  0.0052, -1.1748]],
       
               [[-0.9243, -0.2126, -1.3020],
                [ 1.5325,  0.0052, -1.1748]],
       
               [[-0.9243, -0.2126, -1.3020],
                [ 1.5325,  0.0052, -1.1748]],
       
               [[-0.9243, -0.2126, -1.3020],
                [ 1.5325,  0.0052, -1.1748]],
       
               [[-0.9243, -0.2126, -1.3020],
                [ 1.5325,  0.0052, -1.1748]],
       
               [[-0.9243, -0.2126, -1.3020],
                [ 1.5325,  0.0052, -1.1748]]]),
    x: Batch(
           c: tensor([[[-0.7799,  0.6385, -0.5867,  1.0679],
                       [-0.9532,  0.0329, -0.7299,  1.0554],
                       [-0.6603, -0.0979,  0.8918,  1.2448]],
         

In [26]:
%timeit Batch.stack(batches)

77 µs ± 337 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9243, -0.2126, -1.3020],
               [ 1.5325,  0.0052, -1.1748],
               [-0.9243, -0.2126, -1.3020],
               [ 1.5325,  0.0052, -1.1748],
               [-0.9243, -0.2126, -1.3020],
               [ 1.5325,  0.0052, -1.1748],
               [-0.9243, -0.2126, -1.3020],
               [ 1.5325,  0.0052, -1.1748],
               [-0.9243, -0.2126, -1.3020],
               [ 1.5325,  0.0052, -1.1748],
               [-0.9243, -0.2126, -1.3020],
               [ 1.5325,  0.0052, -1.1748],
               [-0.9243, -0.2126, -1.3020],
               [ 1.5325,  0.0052, -1.1748],
               [-0.9243, -0.2126, -1.3020],
               [ 1.5325,  0.0052, -1.1748]]),
    x: Batch(
           c: tensor([[-0.7799,  0.6385, -0.5867,  1.0679],
                      [-0.9532,  0.0329, -0.7299,  1.0554],
                      [-0.6603, -0.0979,  0.8918,  1.2448],
                      [-0.7799,  0.6385, -0.5867,  1.0679],
                      [-0.9532,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 409 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

326 µs ± 7.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
